# AI Music Post-Production Pipeline

This notebook implements the 7-step post-production workflow for AI-generated music. It is designed to run directly in Google Colab and integrates with your Google Drive for storing models, inputs, and outputs.

### Setup on Your Google Drive (One-time only)

1.  Create a main folder in your Google Drive named `AI_Music_Pipeline`.
2.  Inside that folder, create another folder named `inputs`.
3.  Place the song you want to process inside the `AI_Music_Pipeline/inputs/` folder.
4.  Confirm your RVC model `G_8200.pth` and its corresponding `.index` file are located in `/MyDrive/models/RVC/`.

## Cell 1: Mount Google Drive

Run this cell first. It will prompt you to authorize access to your Google Drive, making your files available to this notebook.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Cell 2: Environment Setup

Run this cell second. It will take a few minutes to install all necessary system packages and Python libraries, and clone the required repositories like RVC-WebUI.

In [ ]:
# 0. Upgrade packaging tools\n!pip install --upgrade pip==24.0 setuptools wheel\n\n# 1. Install system packages and build tools
!apt-get update
!apt-get install -y --no-install-recommends ffmpeg lv2file liblilv-dev rubberband-cli git build-essential
!apt-get install -y lsp-plugins-lv2

# 2. Clone and install Airwindows LV2 plugins
!rm -rf airwindows-lv2
!git clone https://github.com/hannesbraun/airwindows-lv2.git
%cd airwindows-lv2
!make install
%cd ..

# 3. Clone RVC-WebUI and install its dependencies
!rm -rf Retrieval-based-Voice-Conversion-WebUI
!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git
%cd Retrieval-based-Voice-Conversion-WebUI
!sed -i '/torch/d' requirements.txt
!sed -i '/torchaudio/d' requirements.txt
!sed -i '/tensorboard/d' requirements.txt
!pip install -r requirements.txt --quiet
%cd ..

# 4. Install Python packages
!pip install --upgrade --quiet --break-system-packages \
    BS-RoFormer \
    "pedalboard>=0.8.6" \
    pyloudnorm \
    matchering==2.0.6 \
    soundfile \
    librosa \
    ffmpeg-python

# 5. Set LV2_PATH environment variable for plugins to be found
import os
os.environ['LV2_PATH'] = '/root/.lv2:/usr/lib/lv2:/usr/local/lib/lv2'

print("✅ Environment setup complete.")

## Cell 3: Imports and Configuration

This cell imports all necessary Python libraries and sets up the file paths for the pipeline. 

**🚨 ACTION REQUIRED:** You must edit the variables in the `--- Main Configuration ---` section below to match your filenames.

In [ ]:
import os
import subprocess
import soundfile as sf
import pyloudnorm as pyln
import numpy as np
from pedalboard import Pedalboard, Compressor, Gain, Limiter, load_plugin
from pedalboard.io import AudioFile
import matchering as mg
import ffmpeg
import requests
import shutil

# --- Main Configuration (Edit these paths) ---

DRIVE_PIPELINE_DIR = "/content/drive/MyDrive/AI_Music_Pipeline"
INPUTS_DIR = os.path.join(DRIVE_PIPELINE_DIR, "inputs")
OUTPUT_DIR = os.path.join(DRIVE_PIPELINE_DIR, "outputs")
RVC_MODEL_PATH = "/content/drive/MyDrive/models/RVC/G_8200.pth"
RVC_INDEX_PATH = "/content/drive/MyDrive/models/RVC/added_G_8200.index"
RVC_PITCH_SHIFT = 0
REF_FILENAME_WAV = "ref_music.wav"

# --- Directory Setup (No edits needed below this line) ---
STEMS_DIR = os.path.join(OUTPUT_DIR, "1_stems")
RVC_DIR = os.path.join(OUTPUT_DIR, "2_rvc_vocals")
PROCESSED_DIR = os.path.join(OUTPUT_DIR, "3_processed_stems")
MIX_DIR = os.path.join(OUTPUT_DIR, "4_mixdown")
MASTER_DIR = os.path.join(OUTPUT_DIR, "5_master")

print("✅ Configuration loaded.")

## Cell 4: Pipeline Helper Functions

This cell defines helper functions for creating directories and downloading the reference track. No edits are needed here.

In [ ]:
def setup_directories():
    # Also create the main input directory if it doesn't exist
    os.makedirs(INPUTS_DIR, exist_ok=True)
    # Create a directory to move completed source files to
    COMPLETED_DIR = os.path.join(OUTPUT_DIR, "completed_sources")
    for path in [OUTPUT_DIR, STEMS_DIR, RVC_DIR, PROCESSED_DIR, MIX_DIR, MASTER_DIR, COMPLETED_DIR]:
        os.makedirs(path, exist_ok=True)

def download_reference_track():
    drive_ref_path = os.path.join(DRIVE_PIPELINE_DIR, "inputs", "ref_music.wav")
    local_ref_path = "ref_music.wav"
    if not os.path.exists(drive_ref_path):
        raise FileNotFoundError(f"Reference file not found at {drive_ref_path}. Please upload 'ref_music.wav' to the 'AI_Music_Pipeline/inputs' folder on your Google Drive.")
    print(f"Copying reference track from {drive_ref_path} to {local_ref_path}")
    shutil.copy(drive_ref_path, local_ref_path)
    print(f"Reference track ready: {local_ref_path}")

def check_environment():
    """Checks if all required command-line tools are available."""
    print("Checking for required tools...")
    if not shutil.which("bs_roformer"):
        raise FileNotFoundError("The 'bs_roformer' command was not found. This means the environment is not set up correctly. Please run Cell 2 (Environment Setup) and wait for it to complete before running the pipeline.")
    print("✅ All required tools found.")

print("✅ Helper functions defined.")

## Cell 5: Core Pipeline Implementation

This cell contains all the core logic for the 7-step audio processing pipeline. No edits are needed here.

In [ ]:
def separate_stems(input_file, output_dir):
    print("--- Step 1: Separating audio into 4 stems with BS-RoFormer ---")
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Input song '{input_file}' not found. Please make sure it's in the 'inputs' folder on your Drive and the filename is correct.")
    cmd = ["bs_roformer", "--input", input_file, "--output_dir", output_dir, "--model_type", "bs_roformer_hq_ep_300"]
    subprocess.run(cmd, check=True, capture_output=True, text=True)
    print("✅ Stems separated successfully.")

def replace_vocals_rvc(stems_dir, output_dir):
    print("--- Step 2: Replacing vocals using RVC ---")
    vocals_in = os.path.join(stems_dir, "vocals.wav")
    vocals_out = os.path.join(output_dir, "vocals_rvc.wav")
    if not os.path.exists(RVC_MODEL_PATH) or not os.path.exists(RVC_INDEX_PATH):
        print(f"⚠️ RVC model or index file not found on your Google Drive. Skipping vocal replacement. Searched for {RVC_MODEL_PATH} and {RVC_INDEX_PATH}")
        shutil.copy(vocals_in, vocals_out)
        return
    rvc_script = "/content/RVC-WebUI/tools/infer_cli.py"
    cmd = ["python", rvc_script, "--f0up_key", str(RVC_PITCH_SHIFT), "--input_path", vocals_in, "--index_path", RVC_INDEX_PATH, "--f0method", "rmvpe", "--model_path", RVC_MODEL_PATH, "--output_path", vocals_out, "--index_rate", "0.75", "--filter_radius", "3", "--resample_sr", "48000", "--rms_mix_rate", "0.25", "--protect", "0.33"]
    print("Executing RVC inference...")
    result = subprocess.run(cmd, check=True, capture_output=True, text=True)
    print(result.stdout)
    print(result.stderr)
    print("✅ Vocal replacement successful.")

def run_ffmpeg_chain(infile, outfile, filter_chain_list):
    filter_chain_str = ",".join(filter_chain_list)
    (ffmpeg.input(infile).output(outfile, af=filter_chain_str, ar=48000).overwrite_output().run(quiet=True))

def process_all_stems(rvc_dir, stems_dir, processed_dir):
    print("--- Step 3: Cleaning and processing all stems ---")
    
    # Vocals
    print("Processing vocals...")
    vocals_in = os.path.join(rvc_dir, "vocals_rvc.wav")
    vocals_out = os.path.join(processed_dir, "vocals.wav")
    vocal_ffmpeg_filters = [
        "firequalizer=gain='if(f<90,-18, if(f>16000,-15,0))':zero_phase=1",
        "afftdn=nf=-25",
        "anequalizer=f=250:w=2:g=-3",
        "anequalizer=f=3000:w=2:g=+2",
        "anequalizer=f=12000:w=1.5:g=+3",
        "acompressor=threshold=-18dB:ratio=3:attack=10:release=120"
    ]
    temp_vocals = os.path.join(processed_dir, "temp_vocals.wav")
    run_ffmpeg_chain(vocals_in, temp_vocals, vocal_ffmpeg_filters)
    deesser = load_plugin("http://lsp-plug.in/plugins/deesser_stereo")
    limiter = load_plugin("http://lsp-plug.in/plugins/fast_limiter_stereo")
    limiter.limit = -0.5
    vocal_board = Pedalboard([deesser, limiter])
    with AudioFile(temp_vocals, 'r') as f:
        with AudioFile(vocals_out, 'w', f.samplerate, f.num_channels) as o:
            o.write(vocal_board(f.read(f.frames)))
    os.remove(temp_vocals)

    # Drums
    print("Processing drums...")
    drums_in = os.path.join(stems_dir, "drums.wav")
    drums_out = os.path.join(processed_dir, "drums.wav")
    drum_ffmpeg_filters = ["highpass=f=45", "lowpass=f=18000", "anequalizer=f=400:w=3:g=-3", "acompressor=ratio=2:threshold=-12dB"]
    temp_drums = os.path.join(processed_dir, "temp_drums.wav")
    run_ffmpeg_chain(drums_in, temp_drums, drum_ffmpeg_filters)
    saturator = load_plugin("http://lsp-plug.in/plugins/saturator_stereo")
    saturator.drive = 2.0
    drum_board = Pedalboard([saturator])
    with AudioFile(temp_drums, 'r') as f:
        with AudioFile(drums_out, 'w', f.samplerate, f.num_channels) as o:
            o.write(drum_board(f.read(f.frames)))
    os.remove(temp_drums)

    # Bass
    print("Processing bass...")
    bass_in = os.path.join(stems_dir, "bass.wav")
    bass_out = os.path.join(processed_dir, "bass.wav")
    bass_ffmpeg_filters = ["highpass=f=30", "lowpass=f=8000", "acompressor=threshold=-15dB:ratio=4"]
    temp_bass = os.path.join(processed_dir, "temp_bass.wav")
    run_ffmpeg_chain(bass_in, temp_bass, bass_ffmpeg_filters)
    basskit = load_plugin("http://hannesbraun.de/plugins/airwindows/BassKit")
    basskit.Boost = 0.3
    bass_board = Pedalboard([basskit])
    with AudioFile(temp_bass, 'r') as f:
        with AudioFile(bass_out, 'w', f.samplerate, f.num_channels) as o:
            o.write(bass_board(f.read(f.frames)))
    os.remove(temp_bass)

    # Other
    print("Processing 'other' stem...")
    other_in = os.path.join(stems_dir, "other.wav")
    other_out = os.path.join(processed_dir, "other.wav")
    other_ffmpeg_filters = ["highpass=f=110", "lowpass=f=16000", "anequalizer=f=5000:w=2:g=+2", "acompressor=threshold=-14dB:ratio=2"]
    run_ffmpeg_chain(other_in, other_out, other_ffmpeg_filters)
    print("✅ Stems processed successfully.")

def level_stems(processed_dir):
    print("--- Step 4: Leveling stems to target LUFS & peak ---")
    stems_to_level = { "vocals": {"target_lufs": -18.0, "target_peak": -6.0}, "drums":  {"target_lufs": -14.0, "target_peak": -3.0}, "bass":   {"target_lufs": -17.0, "target_peak": -5.0}, "other":  {"target_lufs": -20.0, "target_peak": -7.0} }
    meter = pyln.Meter(48000)
    for name, targets in stems_to_level.items():
        filepath = os.path.join(processed_dir, f"{name}.wav")
        data, rate = sf.read(filepath)
        loudness = meter.integrated_loudness(data)
        gain_db = targets["target_lufs"] - loudness
        board = Pedalboard([Gain(gain_db=gain_db)])
        leveled_data = board(data, rate)
        peak_linear = np.max(np.abs(leveled_data))
        target_peak_linear = 10**(targets["target_peak"] / 20.0)
        if peak_linear > target_peak_linear:
            leveled_data *= (target_peak_linear / peak_linear)
        sf.write(filepath, leveled_data.T, rate)
        final_lufs = pyln.Meter(rate).integrated_loudness(leveled_data.T)
        final_peak = 20 * np.log10(np.max(np.abs(leveled_data)))
        print(f"  Leveled '{name}': LUFS={final_lufs:.2f}, Peak={final_peak:.2f} dBFS")
    print("✅ Stems leveled successfully.")

def mix_stems(processed_dir, mix_dir):
    print("--- Step 5: Mixing all stems ---")
    stem_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.endswith('.wav')]
    stems_audio, max_len, samplerate = [], 0, 48000
    for f in stem_files:
        with AudioFile(f) as af:
            stems_audio.append(af.read(af.frames))
            if af.frames > max_len: max_len = af.frames
            samplerate = af.samplerate
    for i, stem in enumerate(stems_audio):
        if stem.shape[1] < max_len:
            stems_audio[i] = np.concatenate([stem, np.zeros((stem.shape[0], max_len - stem.shape[1]))], axis=1)
    mixdown = np.sum(np.array(stems_audio), axis=0)
    peak, target_peak = np.max(np.abs(mixdown)), 10**(-4.0 / 20.0)
    if peak > target_peak: mixdown *= (target_peak / peak)
    mix_file = os.path.join(mix_dir, "mixdown.wav")
    with AudioFile(mix_file, 'w', samplerate, mixdown.shape[0]) as f: f.write(mixdown)
    print(f"✅ Mixdown saved to {mix_file}")
    return mix_file

def master_and_export(mix_file, master_dir, ref_wav):
    print("--- Step 6 & 7: Mastering with Matchering and exporting final WAV ---")
    master_file = os.path.join(master_dir, "master_24bit_48kHz.wav")
    mg.process(target=mix_file, reference=ref_wav, results=[mg.pcm24(master_file)], sample_rate=48000)
    data, rate = sf.read(master_file)
    meter = pyln.Meter(rate)
    loudness = meter.integrated_loudness(data)
    peak = 20 * np.log10(np.max(np.abs(data)))
    print("✅ Mastering complete!")
    print(f"Final output: {master_file}")
    print(f"  - Format: 24-bit, {rate} Hz WAV")
    print(f"  - Integrated Loudness: {loudness:.2f} LUFS (Target: ~-13 LUFS)")
    print(f"  - Peak: {peak:.2f} dBFS (Target: -1 dBTP)")

print("✅ Core pipeline functions defined.")

## Cell 6: Execute the Pipeline

Run this final cell to start the entire process.

In [ ]:
def main():
    # --- 1. Find audio files to process ---
    SUPPORTED_EXTENSIONS = ['.wav', '.mp3', '.flac', '.aiff', '.ogg']
    audio_files = []
    for filename in os.listdir(INPUTS_DIR):
        if any(filename.lower().endswith(ext) for ext in SUPPORTED_EXTENSIONS) and filename != REF_FILENAME_WAV:
            audio_files.append(os.path.join(INPUTS_DIR, filename))

    if not audio_files:
        print(f"No audio files found in {INPUTS_DIR}. Nothing to process.")
        return

    print(f"Found {len(audio_files)} song(s) to process:")
    for f in audio_files:
        print(f"  - {os.path.basename(f)}")

    # --- 2. Setup main directories ---
    try:
        check_environment() # Check for missing tools before we start
        setup_directories()
        download_reference_track()
        print("\n--- Starting Batch Processing ---")

        for i, song_path in enumerate(audio_files):
            song_name = os.path.basename(song_path)
            print(f"\n--- Processing Song {i+1}/{len(audio_files)}: {song_name} ---")

            # Run the full pipeline for the current song
            separate_stems(song_path, STEMS_DIR)
            replace_vocals_rvc(STEMS_DIR, RVC_DIR)
            process_all_stems(RVC_DIR, STEMS_DIR, PROCESSED_DIR)
            level_stems(PROCESSED_DIR)
            mix_file = mix_stems(PROCESSED_DIR, MIX_DIR)
            master_and_export(mix_file, MASTER_DIR, REF_FILENAME_WAV)

            # Move the original file to the completed directory
            completed_dir = os.path.join(OUTPUT_DIR, "completed_sources")
            shutil.move(song_path, os.path.join(completed_dir, song_name))
            print(f"Moved '{song_name}' to completed sources directory.")
            print(f"--- Finished processing {song_name} ---")

        print("\n🎉🎉🎉 Batch processing finished successfully! 🎉🎉🎉")
        print(f"Find your final mastered track in '{MASTER_DIR}' on your Google Drive.")
    except Exception as e:
        print(f"\n❌ An error occurred during the pipeline: {e}")
        import traceback
        traceback.print_exc()

main()